<br><br><br><br><br>

# Numpy and Pandas

In [ ]:
# Start with a plausible problem: analyze a dataset of daily Newark temperatures since 1883.

with open("data/newark-temperature-avg.txt") as file:
    print(file.readlines())

In [3]:
# Start with a plausible problem: analyze a dataset of daily Newark temperatures since 1883.

temperatures = []
with open("data/newark-temperature-avg.txt") as file:
    for line in file.readlines():
        temperatures.append(float(line))

print("how many?", len(temperatures))
print("starts with", temperatures[:10])
print("ends with  ", temperatures[-10:])

how many? 42019
starts with [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
ends with   [43.0, 47.0, 49.0, 52.0, 48.0, 52.0, 62.0, 68.0, 59.0, 47.0]
